# Data cleaning

In [1]:
import pandas as pd
import numpy as np # Numerical computation package
np.random.seed(1) # Set the random seed for reproduceability

ped = pd.read_csv("köln-schildergasse (west)-20180430-20200131-hour.csv", sep = ";")
holi = pd.read_csv("Feiertage_2018.01.01_2020.01.31.csv", sep = ";")
##########################################print(ped.head(3))
##########################################print(holi.head(3))
#test


#########################################################################
#Machen wir das in eine Neue Box oder nicht? Die Print sachen können wir sonst raus nehmen?
#########################################################################


#Splitting up the Time of measurement into date and time
ped[["date", "time"]] = ped["time of measurement"].str.split(" ", n=1, expand=True)
#Drpo Time of measuremnt, because it is split now and drop location
ped = ped.drop(["time of measurement", "location"], axis=1)
#Drop incidents, if only nan
ped.dropna(how='all', axis=1, inplace=True)

# restructure by date
ped = ped[ ["date", "time"] + [ col for col in ped.columns if col != "date" and col!= "time"] ]
#change name
holi = holi.rename(columns={"name": "holiday"})
#merge on date
ped = pd.merge(ped, holi, how = "left")

#put hollyday as 1 and no holyday as 0
ped["holiday"] = ped["holiday"].fillna(0)
ped["holiday"] = ped["holiday"].where(ped["holiday"] == 0, 1)

# clear time from the minute variation
ped["time"] = ped["time"].str.slice_replace(8, 16)

# Get a better insight into the weather data and missing data
##########################################print(ped["weather condition"].unique())
##########################################print(ped.isnull().sum() / ped.shape[0])

#See where the data are missing
null_data = ped[ped.isnull().any(axis=1)]
##########################################print(null_data)


#########################################################################
#Machen wir das in eine Neue Box oder nicht? Die Print sachen können wir sonst raus nehmen?
#########################################################################


#because the data for temperature and weather condition are both missing, this can be dropped, due to the huge sample size
ped.dropna(inplace=True)

#Import datapackages
from datetime import date, timedelta
from datetime import datetime

#Import Dataframe from the holidays
school_holi = pd.read_csv("OpenData_Ferientermine.csv", sep = ";")
days_df_list = []

#Create dataframe with all dates singular listed 
for i in range(school_holi.shape[0]):
    #Get the start and enddate from the holidays
    start_date = datetime.strptime(school_holi["ErsterTagDate"][i], "%d.%m.%Y").date()
    end_date = datetime.strptime(school_holi["LetzterTagDate"][i], "%d.%m.%Y").date()

    #create dataframes out of the range and append to a list
    delta = end_date - start_date
    days = [start_date + timedelta(days=n) for n in range(delta.days + 1)]
    days_df = pd.DataFrame (days, columns = ['date'])
    days_df_list.append(days_df)

#create single dataframe out of the list and add value 1 for holiday
new_holi = pd.concat(days_df_list).reset_index(drop=True)
new_holi["school holiday"] = 1
new_holi["date"] = new_holi["date"].astype(str)

#merg of the two frames
ped = pd.merge(ped, new_holi, how = "left", on="date")
ped["school holiday"] = ped["school holiday"].fillna(0)

#change float to int
ped["school holiday"] = np.int64(ped["school holiday"].astype(int))

#Get Column with months
ped["months"] = ped["date"].str.slice(5, 7)

#Transform time to hours
ped["hour"] = ped["time"].str.slice(0, 2)


# Save for the API
ped.to_csv("data_clean_without_dummies.csv", sep = ";")


#get dummies for models
ped = pd.get_dummies(ped, columns=["weekday", "weather condition", "months", "hour"])


#########################################################################
#Machen wir das in eine Neue Box oder nicht? Die Print sachen können wir sonst raus nehmen?
#########################################################################

#Creating the PCA and look if we can safe data storage
from sklearn.decomposition import PCA

# For PCA, we want to standardize our data, i.e., de-mean it and divide it by the standard deviation
standardize = lambda x: (x - x.mean()) / x.std()
ped["temperature in ºc"] = standardize(ped["temperature in ºc"])


###################################################################################
#Dieser Teil ist für die PCA (klappt das alles?), Die PC sind alle hinten dran gehängt
###################################################################################
#Safe all features to calculate the cpa
features = ped.columns[3:] # Features

# Column names for our principal components
principal_components = [f"PC{i+1}" for i in range(len(features))]

# Run PCA with 4 principal components on our iris data
pca = PCA(n_components=len(features))
ped[principal_components] = pca.fit_transform(ped[features])


###################################################################################
#57 sind die Columns, ab dem pca los geht und k giebt an, wie viele benötigt werden für die Schwelle
###################################################################################
threshhold = 0.99
k = np.argmax(pca.explained_variance_ratio_.cumsum() >= threshhold)
print(k)
print(ped.columns[57:])
###################################################################################

#save prepeared data
ped.to_csv("data_clean_with_dummies.csv")



46
Index(['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10',
       'PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16', 'PC17', 'PC18', 'PC19',
       'PC20', 'PC21', 'PC22', 'PC23', 'PC24', 'PC25', 'PC26', 'PC27', 'PC28',
       'PC29', 'PC30', 'PC31', 'PC32', 'PC33', 'PC34', 'PC35', 'PC36', 'PC37',
       'PC38', 'PC39', 'PC40', 'PC41', 'PC42', 'PC43', 'PC44', 'PC45', 'PC46',
       'PC47', 'PC48', 'PC49', 'PC50', 'PC51', 'PC52', 'PC53', 'PC54'],
      dtype='object')
